Так как в задании требуется сделать легковесный классификатор, сразу откажемся от идеи использовать нейронные сети, ведь если не требуется state-of-the-art результата даже линейные модели показывают себя неплохо на задачах классификации текста.

# Обработка данных

In [9]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import ComplementNB
from sklearn.model_selection import cross_val_score

In [14]:
train = pd.read_csv('./train.csv').fillna(' ')
test = pd.read_csv('./test.csv').fillna(' ')
train_text = train['comment_text']
test_text = test['comment_text']
class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

Для обработки текста будем использовать tf-idf на биграммах.

In [11]:
vectorizer = TfidfVectorizer(ngram_range=(1,2),
                min_df=3, max_df=0.9, strip_accents='unicode',
                sublinear_tf=1, token_pattern=r'\w{1,}', stop_words='english')
X = vectorizer.fit_transform(train_text)
X_test = vectorizer.transform(test_text)

In [7]:
tfidf_train_text = vectorizer.fit_transform(train_text)
tfidf_test_text = vectorizer.transform(test_text)

# Наивный байес

Для начала попробуем решить данную задачу при помощи наивного байеса, который является самой легковесной моделью

In [20]:
def fit_model(clf):
    scores = []
    for class_name in class_names:
        y = train[class_name]
        score = np.mean(cross_val_score(clf, X, y, scoring='roc_auc', cv=3))
        scores.append(score)
    print("Total score: {}".format(np.mean(scores)))

In [22]:
bayes_clf = ComplementNB()
fit_model(bayes_clf)

Total score: 0.8566646243664794


Как мы можем видеть даже такая простая модель как наивный байес дает достаточно хорошие результаты, однако стоит попытаться добиться лучшего качества при помощи логистической регрессии.

# Логистическая регрессия

In [43]:
lr_clf = LogisticRegression(dual=True, solver='liblinear')

In [44]:
fit_model(lr_clf)

Total score: 0.9794040139355179


По сравнению с предыдущей моделью качество заметно возрасло, однако можно ли повысить его еще больше не особо усложняя модель?

# NB-SVM

Построим нашу модель совместив предыдущие две, основываясь на данной статье https://nlp.stanford.edu/pubs/sidaw12_simple_sentiment.pdf

In [45]:
scores = []
preds = np.zeros((len(test), len(class_names)))

In [49]:
submission = pd.DataFrame.from_dict({'id': test['id']})

final_model = []

for i, class_name in enumerate(class_names):
    y = train[class_name]
    
    bayes_clf.fit(X, y)
    log_probas = bayes_clf.feature_log_prob_
    r = log_probas[-1]/log_probas[0]
    X_b = X.multiply(r)
    
    model = LogisticRegression(dual=True, solver='liblinear')
    
    score = np.mean(cross_val_score(model, X_b, y, scoring='roc_auc', cv=3))
    scores.append(score)
    
    model.fit(X_b, y)
    final_model.append(model)
    
    submission[class_name] = lr_clf.predict_proba(X_test)[:, 1]
    
print("Total score: {}".format(np.mean(scores)))

Total score: 0.9810456672592762


Как можем видеть, качество нашей модели еще немного возрасло, составим теперь посылку на kaggle.

In [48]:
submission.to_csv('submission.csv', index=False)

Итоговое качество нашей модели можем видеть здесь:
![alt text](score.png)

# Отчет

Улучшить данную модель можно при помощи подбора параметра регуляризации у логистической регрессии, а также поигравшись с параметрами tf-idf.

Для того чтобы облегчить нашу модель мы можем полностью отказаться от использований фичей из наивного байеса, однако это приводит к падению public score на процент. Также облегчить модель поможет сокращение максимального количества фичей в tf-idf.

Баланс между качеством и скоростью выбирается в зависимости от задачи, зачастую нам не нужна идеальная модель, поэтому мы можем сэкономить в ее весе, однако всегда стоит следить за падением качества. Проверить гипотезы о том что же нам лучше, можно на пользователях, например, спрашивая их довольны ли они точностью или скоростью нашего сервиса. Основываясь на данных результатах мы можем понять, какую модель предпочтительнее использовать.

Зачастую гораздо рациональнее использовать легкие модели для начального решения задач, а потом уже смотреть устраивает ли нас результат. Разумеется, есть задачи в которых нельзя применить логистическую регрессию, однако даже в них разумно использовать легкие нейронные сети, хотя бы для какого то начального приближения. При этом есть задачи в которых тяжелые модели в разы лучше всех остальных, именно в этих случаях полезно пытаться оптимизировать уже хорошо показавшие себя тяжеловесные модели.

Недостатком нашей модели может являться то, что на более серьезных задачах качество наверняка заметно упадет. Поэтому без нейронных сетей обойтись не получится.

# Бонус: Интерпретируемость

Посмотрим на то, какие слова наша модель считала признаком того, что комментарий является `threat`.

In [78]:
vocab = vectorizer.vocabulary_

In [94]:
idx_to_word = {vocab[word]:word for word in vocab}
inds = final_model[3].coef_.squeeze().argsort()[-15:][::-1]
words = [idx_to_word[ind] for ind in inds]
words

['kill',
 'die',
 'fucking',
 'death',
 'rape',
 'ass',
 'going',
 'burn',
 'fuck',
 'shoot',
 'destroy',
 'hope die',
 'dead',
 'hope',
 'hang']

Это было просто, ведь логистическая регрессия очень хорошо интерпретируется. Те слова, которым присвоены наибольшие веса и влияют на итоговый вердикт нашей модели.